In [0]:
from pyspark.sql import functions as F

TODAY = "2025-05-21"

# Input from Bronze
BRONZE_PATH = f"dbfs:/bronze/weather/{TODAY}"

# Output Silver path
SILVER_PATH = f"dbfs:/silver/weather/{TODAY}"


In [0]:
df_bronze = spark.read.format("delta").load(BRONZE_PATH)
df_bronze.select("name", "forecast_time", "temperature", "shortForecast", "windSpeed").show(5, truncate=False)


In [0]:
df_silver = (
    df_bronze.select(
        F.col("forecast_time"),
        F.col("temperature").cast("int"),
        F.col("shortForecast").alias("condition"),
        F.col("windSpeed"),
        F.col("forecast_retrieved_at")
    )
    .withColumn("processed_at", F.current_timestamp())
    .dropDuplicates(["forecast_time"])
)


In [0]:
df_silver.write.format("delta").mode("overwrite").save(SILVER_PATH)
print("✓ Weather Silver table saved")
